Add additional package `ortools` (missing in the Colab defaults)

In [1]:
!pip install ortools

Import libraries (Colab defaults are used)

In [2]:
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import io
import pandas as pd

Read dataset as a Pandas dataframe

_Notice that the data used for the teams is partially accurate with the qualified teams up to early July 2024. The exact list will only be available in September after the playoff rounds will be completed! The list chosen in this notebook for the non qualified teams is purely random!_

In [3]:
teams_df = pd.read_csv(io.StringIO('''
Team,Association,Tier
Manchester City,England,1
Real Madrid,Spain,1
Bayern Munich,Germany,1
Liverpool,England,1
Paris Saint-Germain,France,1
Borussia Dortmund,Germany,1
Inter,Italy,1
Bayer Leverkusen,Germany,1
RB Leipzig,Germany,1
Benfica,Portugal,2
Atlético Madrid,Spain,2
Barcelona,Spain,2
Arsenal,England,2
Atalanta,Italy,2
Milan,Italy,2
Juventus,Italy,2
Club Bruges,Belgium,2
Feyenoord,Netherlands,2
PSV,Netherlands,3
Slavia Prague,Czech Republic,3
Sporting CP,Portugal,3
Lille,France,3
Dinamo Zagreb,Croatia,3
Shakhtar Donetsk,Ukraine,3
Red Star Belgrade,Serbia,3
Young Boys,Switzerland,3
Union SG,Belgium,3
Celtic,Scotland,4
VfB Stuttgart,Germany,4
Aston Villa,England,4
Girona,Spain,4
Galatasaray,Turkey,4
Brest,France,4
Twente,Netherlands,4
Bologna,Italy,4
Sturm Graz,Austria,4
'''), header=0)

teams_df['Association_Number'] = teams_df['Association'].astype('category').cat.codes.astype(int)
teams_df = teams_df.sort_values(by=['Tier', 'Association_Number'])
teams_df = teams_df.reset_index(drop=True)

In [4]:
teams_df.head(10)

,Team,Association,Tier,Association_Number
0,Manchester City,England,1,4
1,Liverpool,England,1,4
2,Paris Saint-Germain,France,1,5
3,Bayern Munich,Germany,1,6
4,Borussia Dortmund,Germany,1,6
5,Bayer Leverkusen,Germany,1,6
6,RB Leipzig,Germany,1,6
7,Inter,Italy,1,7
8,Real Madrid,Spain,1,12
9,Club Bruges,Belgium,2,1


Let's introduce all the necessary input variables

In [5]:
# Number of teams
N_TEAMS = teams_df.shape[0]

# Tiers
N_TIERS = 4

# Matches
N_MATCHES = 8

assert N_TEAMS % N_TIERS == 0

TIERS = {}

n_teams_per_tier = N_TEAMS / N_TIERS

for k in range(N_TIERS):
  TIERS[k] = list(range(int(k * n_teams_per_tier), int((k + 1) * n_teams_per_tier)))


In [6]:
TEAMS = teams_df['Team'].tolist()
ASSOCIATIONS = {}

for idx, row in teams_df.iterrows():
  if row['Association_Number'] not in ASSOCIATIONS.keys():
    ASSOCIATIONS[row['Association_Number']] = [idx]
  else:
    ASSOCIATIONS[row['Association_Number']].append(idx)

In [7]:
ASSOCIATIONS

{4: [0, 1, 10, 28],
 5: [2, 21, 29],
 6: [3, 4, 5, 6, 30],
 7: [7, 11, 12, 13, 31],
 12: [8, 16, 17, 34],
 1: [9, 18],
 8: [14, 22, 32],
 9: [15, 23],
 2: [19],
 3: [20],
 11: [24],
 13: [25],
 15: [26],
 0: [27],
 10: [33],
 14: [35]}

In [8]:
len(TEAMS)

36

In [9]:
TIERS

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8],
 1: [9, 10, 11, 12, 13, 14, 15, 16, 17],
 2: [18, 19, 20, 21, 22, 23, 24, 25, 26],
 3: [27, 28, 29, 30, 31, 32, 33, 34, 35]}

Let's formulate the problem by defining decision variables and the necessary constraints

In [10]:
model = cp_model.CpModel()

# x decision variables - who plays against each other
x = {i:
     {j:
      {
        t: model.NewBoolVar(f'x_{i}_{j}_{t}')
        for t in range(N_MATCHES)
      }
      for j in range(N_TEAMS)
     }
     for i in range(N_TEAMS)
    }

# y decision variables - who plays at home each match
y = {i:
     {j:
      {
        t: model.NewBoolVar(f'y_{i}_{j}_{t}')
        for t in range(N_MATCHES)
      }
      for j in range(N_TEAMS)
     }
     for i in range(N_TEAMS)
    }

# fundamental constraints ensuring there is consistency in the definition of the decision variables
for i in range(N_TEAMS):
  for t in range(N_MATCHES):
    model.Add(x[i][i][t] == 0)
    for j in range(i + 1, N_TEAMS):
      model.Add(x[i][j][t] == x[j][i][t])

# every team has 1 and only 1 match every day
for i in range(N_TEAMS):
  for t in range(N_MATCHES):
    model.Add(sum([x[i][j][t] for j in range(N_TEAMS)]) == 1)

# every team plays every other team at maximum 1 time
for i in range(N_TEAMS):
  for j in range(N_TEAMS):
    model.Add(sum([x[i][j][t] for t in range(N_MATCHES)]) <= 1)

# every team plays equal number of matches with teams from every tier
for i in range(N_TEAMS):
  for k in range(N_TIERS):
    model.Add(sum([x[i][j][t] for j in TIERS[k] for t in range(N_MATCHES)]) == int(N_MATCHES / N_TIERS))

# in every match, only 1 team can play at home
for i in range(N_TEAMS):
  for t in range(N_MATCHES):
    for j in range(i + 1, N_TEAMS):
      model.Add((x[i][j][t] + x[j][i][t]) == 2 * (y[i][j][t] + y[j][i][t]))

# constraints defining every team needs to play exactly half of the games at home (remaining away)
for i in range(N_TEAMS):
  model.Add(sum([y[i][j][t] for j in range(N_TEAMS) for t in range(N_MATCHES)]) == int(N_MATCHES / 2))

# constraints defining that each team cannot play consecutively 4 or more games at home or away -
# NOTE: these constraints are commented out for simplicity - lighter problem to be solved
if False:
  for i in range(N_TEAMS):
    for t in range(N_MATCHES - 3):
      model.Add(sum([y[i][j][t2] for j in range(N_TEAMS) for t2 in range(t, t + 3)]) <= 3)
      model.Add(sum([y[i][j][t2] for j in range(N_TEAMS) for t2 in range(t, t + 3)]) >= 1)

# constraints defining that each team either plays the first or the last game at home
# NOTE: these constraints are commented out for simplicity - lighter problem to be solved
if False:
  for i in range(N_TEAMS):
    model.Add(sum([y[i][j][0] + y[i][j][N_MATCHES - 1] for j in range(N_TEAMS)]) == 1)

# constraints defining that each team cannot play another team from the same association
for association, teams_in_association in ASSOCIATIONS.items():
  for i in teams_in_association:
    model.Add(sum([x[i][j][t] for j in teams_in_association for t in range(N_MATCHES)]) == 0)

Now, solve the problem and measure the time taken to resolution

In [11]:
%%time
# use solver
solver = cp_model.CpSolver()

status = solver.Solve(model)

CPU times: user 28.9 s, sys: 484 ms, total: 29.3 s
Wall time: 26.2 s


Collect the results in a readable format

In [12]:
if status in [cp_model.FEASIBLE, cp_model.OPTIMAL]:
  x_opt = {i:
      {j:
        {
          t: solver.value(x[i][j][t])
          for t in range(N_MATCHES)
        }
        for j in range(N_TEAMS)
      }
      for i in range(N_TEAMS)
      }
  y_opt = {i:
    {j:
      {
        t: solver.value(y[i][j][t])
        for t in range(N_MATCHES)
      }
      for j in range(N_TEAMS)
    }
    for i in range(N_TEAMS)
    }

  matches = {f'Day {t}': [] for t in range(N_MATCHES)}

  for i in range(N_TEAMS):
    for j in range(i + 1, N_TEAMS):
      for t in range(N_MATCHES):
        if x_opt[i][j][t] == 1:
          team_i_printed = f'{TEAMS[i]}({1 + int(i/n_teams_per_tier)})'
          team_j_printed = f'{TEAMS[j]}({1 + int(j/n_teams_per_tier)})'
          if y_opt[i][j][t] == 1:
            matches[f'Day {t}'].append(team_i_printed + ' - ' + team_j_printed)
          elif y_opt[j][i][t] == 1:
            matches[f'Day {t}'].append(team_j_printed + ' - ' + team_i_printed)
          else:
            raise NotImplementedError
else:
  print('Infeasible solution found')

In [13]:
pd.DataFrame(matches)

,Day 0,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
0,Manchester City(1) - Atlético Madrid(2),Brest(4) - Manchester City(1),Bologna(4) - Manchester City(1),Manchester City(1) - Slavia Prague(3),Borussia Dortmund(1) - Manchester City(1),Young Boys(3) - Manchester City(1),Manchester City(1) - RB Leipzig(1),Manchester City(1) - Barcelona(2)
1,Liverpool(1) - Borussia Dortmund(1),Liverpool(1) - Milan(2),Juventus(2) - Liverpool(1),Liverpool(1) - Lille(3),Sporting CP(3) - Liverpool(1),VfB Stuttgart(4) - Liverpool(1),Twente(4) - Liverpool(1),Liverpool(1) - Bayer Leverkusen(1)
2,PSV(3) - Paris Saint-Germain(1),Bologna(4) - Paris Saint-Germain(1),Paris Saint-Germain(1) - Inter(1),Paris Saint-Germain(1) - Arsenal(2),VfB Stuttgart(4) - Paris Saint-Germain(1),Bayer Leverkusen(1) - Paris Saint-Germain(1),Paris Saint-Germain(1) - Atalanta(2),Paris Saint-Germain(1) - Union SG(3)
3,Real Madrid(1) - Bayern Munich(1),Bayern Munich(1) - Atalanta(2),Galatasaray(4) - Bayern Munich(1),Girona(4) - Bayern Munich(1),Benfica(2) - Bayern Munich(1),Bayern Munich(1) - Slavia Prague(3),Bayern Munich(1) - Inter(1),Bayern Munich(1) - Red Star Belgrade(3)
4,Bayer Leverkusen(1) - Barcelona(2),Sporting CP(3) - Borussia Dortmund(1),Borussia Dortmund(1) - Aston Villa(4),Borussia Dortmund(1) - Milan(2),Bayer Leverkusen(1) - Club Bruges(2),Lille(3) - Borussia Dortmund(1),Celtic(4) - Borussia Dortmund(1),Borussia Dortmund(1) - Atlético Madrid(2)
5,Young Boys(3) - RB Leipzig(1),Bayer Leverkusen(1) - Shakhtar Donetsk(3),Union SG(3) - Bayer Leverkusen(1),Twente(4) - Bayer Leverkusen(1),RB Leipzig(1) - Real Madrid(1),Feyenoord(2) - RB Leipzig(1),Girona(4) - Bayer Leverkusen(1),RB Leipzig(1) - Dinamo Zagreb(3)
6,Dinamo Zagreb(3) - Inter(1),RB Leipzig(1) - Juventus(2),Sturm Graz(4) - RB Leipzig(1),RB Leipzig(1) - Brest(4),Red Star Belgrade(3) - Inter(1),Inter(1) - Aston Villa(4),Shakhtar Donetsk(3) - Real Madrid(1),Inter(1) - Arsenal(2)
7,Club Bruges(2) - Feyenoord(2),Inter(1) - Feyenoord(2),Celtic(4) - Real Madrid(1),Inter(1) - Sturm Graz(4),Arsenal(2) - Sturm Graz(4),Real Madrid(1) - Benfica(2),Aston Villa(4) - Club Bruges(2),Real Madrid(1) - Club Bruges(2)
8,Arsenal(2) - Shakhtar Donetsk(3),PSV(3) - Real Madrid(1),Red Star Belgrade(3) - Club Bruges(2),Real Madrid(1) - Galatasaray(4),Atlético Madrid(2) - Atalanta(2),Club Bruges(2) - Girona(4),Juventus(2) - Arsenal(2),Atalanta(2) - Brest(4)
9,Atalanta(2) - VfB Stuttgart(4),Club Bruges(2) - Young Boys(3),Young Boys(3) - Arsenal(2),Club Bruges(2) - Atalanta(2),Aston Villa(4) - Milan(2),Arsenal(2) - Barcelona(2),Milan(2) - Benfica(2),Milan(2) - Sturm Graz(4)
